In [35]:
import cv2
import pytesseract
import os
import numpy as np

In [36]:
!pip install pytesseract

In [37]:
# Set the path to the Tesseract executable file
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 
# ^for this to work, need to install tesseract on laptop at the above path

In [48]:
# Define a function to extract numbers from an image
def extract_numbers(image_path):
    # Load the image
    image = cv2.imread(image_path)
    
#     cv2.imshow('hi', image)
#     cv2.waitKey()
#     cv2.destroyAllWindows()

#     # Convert the image to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Apply thresholding to the image
#     _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

#     # Apply some image processing to improve the OCR accuracy
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
#     thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#     thresh = cv2.medianBlur(thresh, 3)


    # Apply OCR to the image
    text = pytesseract.image_to_string(image, config='--psm 6 outputbase digits')
    # print(text)
    # Extract the numbers from the OCR output
    numbers = [int(s) for s in text.split() if s.isdigit()]

    return numbers

In [50]:
# so hear me out, we get 30 images after applying box detection stuff, we could skip 1st 10 rows, but this is basic code
# where 11-20 are maximum marks row, 21-30 are obtained marks row

# but we dont always get 30 images sooo
# we're going to divide by 3
# I HAVE A LOGIC
# JUST TRUST ME ON THIS ONE
# IT WORKS I PROMISE
maximum_marks = []
obtained_marks = []

results_images_path = "E:/University/Semester 6 - Spring 2023/Digital Image Processing/LabWork/dip_project/AI-DIP_Marks-DC/final/results"
file_list = os.listdir(results_images_path)
num_files = len([f for f in file_list if os.path.isfile(os.path.join(results_images_path, f))])
print(num_files)

# 3x3 kernel for erosion (since it somehow works better???)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))


# Loop over the images and extract the numbers from each image
j = int(num_files/3)
for i in range(1,j+1): # second third of files
    # converting all images to binary first using thresholding for better recognition

    # maybe apply dilation too?
    image_path = f'{results_images_path}/result{i+j}.png'
    img = cv2.imread(image_path, 0)
    (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    
    # applying closing
    img_bin = cv2.morphologyEx(
        img_bin, cv2.MORPH_ERODE, kernel, iterations=1)
        # saving the binary image
    image_bin = f'{results_images_path}/result{i+j}.png'
    cv2.imwrite(image_bin, img_bin)
#     print(image_path)
    image_numbers = extract_numbers(image_bin)
#     print(image_numbers)
    maximum_marks.extend(image_numbers)

# loop to get obtained marks from each question
for i in range(1,j+1):
    image_path = f'{results_images_path}/result{i+(2*j)}.png'
    img = cv2.imread(image_path, 0)
    (thresh, img_bin) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY)
    # applying closing
    img_bin = cv2.morphologyEx(
        img_bin, cv2.MORPH_ERODE, kernel, iterations=1)
    # saving the binary image
    image_bin = f'{results_images_path}/result{i+(2*j)}.png'
    cv2.imwrite(image_bin, img_bin)
    # image_path = f'result{i+21}.png'
    image_numbers = extract_numbers(image_bin)
    obtained_marks.extend(image_numbers)
    
# Print the list of numbers
print(maximum_marks)
print(obtained_marks)

print("total marks: ", sum(maximum_marks))
print("obtained marks: ", sum(obtained_marks))

30
[6, 20, 16, 6]
[6, 30, 16]
total marks:  48
obtained marks:  52
